In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
import mlflow
import mlflow.sklearn
import optuna
import dagshub
import pickle

dagshub.init(repo_owner='satyajeetrai007', repo_name='Youtube-Comment-Sentiment-Analysis', mlflow=True)
mlflow.set_experiment("Algorithms with HyperParameter tuning")

df = pd.read_csv('data_preprocessed.csv').dropna().drop_duplicates()
df.shape

c:\Users\satya\OneDrive\Desktop\youtube-comment-senitment-analysis\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Accessing as satyajeetrai007

Initialized MLflow to track repo "satyajeetrai007/Youtube-Comment-Sentiment-Analysis"

Repository satyajeetrai007/Youtube-Comment-Sentiment-Analysis initialized!

(36243, 2)

In [2]:

# Step 1: Clean and isolate features and target
df = df.dropna(subset=['category'])
X = df['clean_comment']
y = df['category']

# Step 2: Perform the train-test split FIRST to prevent data leakage 🛡️
# 'stratify=y' keeps the same percentage of classes in train and test sets.
X_train_text, X_test_text, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Step 3: Fit the TF-IDF vectorizer ONLY on the training data
vectorizer = TfidfVectorizer(ngram_range=(1, 3), max_features=2000)
X_train_vec = vectorizer.fit_transform(X_train_text)

# Use the already-fitted vectorizer to transform the unseen test data
X_test_vec = vectorizer.transform(X_test_text)

# Step 4: Apply SMOTE to the TRAINING data ONLY to handle class imbalance
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_vec, y_train)

# The test data (X_test_vec, y_test) remains untouched for a valid evaluation.

# --- MLflow and Optuna Functions (no changes needed) ---

def log_mlflow(model_name, model, X_train, X_test, y_train, y_test):
    with mlflow.start_run():
        mlflow.set_tag("mlflow.runName", f"{model_name}_SMOTE_TFIDF_Trigrams_Corrected")
        mlflow.set_tag("experiment_type", "algorithm_comparison")
        mlflow.log_param("algo_name", model_name)

        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        model_filename = f"{model_name}_model.pkl"
        with open(model_filename, "wb") as f:
            pickle.dump(model, f)
        mlflow.log_artifact(model_filename, "model")

# Step 5: Optuna objective function for Random Forest
def objective_rf(trial):
    n_estimators = trial.suggest_int('n_estimators', 50, 300)
    max_depth = trial.suggest_int('max_depth', 3, 20)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 20)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 20)

    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        random_state=42
    )
    # Train on the resampled training data
    model.fit(X_train_resampled, y_train_resampled)
    
    # Evaluate on the original, untouched test data
    y_pred = model.predict(X_test_vec)
    return accuracy_score(y_test, y_pred)

# Step 6: Run Optuna for Random Forest
def run_optuna_experiment():
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_rf, n_trials=30)

    best_params = study.best_params
    best_model = RandomForestClassifier(**best_params, random_state=42)
    
    # Log the best model using the correctly processed datasets
    log_mlflow("RandomForest", best_model, X_train_resampled, X_test_vec, y_train_resampled, y_test)

# Run the experiment
run_optuna_experiment()

[I 2025-08-27 20:24:38,961] A new study created in memory with name: no-name-ace1a21b-3f3e-4646-9485-95928e486d46
[I 2025-08-27 20:24:53,158] Trial 0 finished with value: 0.6842323079045386 and parameters: {'n_estimators': 150, 'max_depth': 14, 'min_samples_split': 4, 'min_samples_leaf': 4}. Best is trial 0 with value: 0.6842323079045386.
[I 2025-08-27 20:24:59,251] Trial 1 finished with value: 0.6702993516347082 and parameters: {'n_estimators': 54, 'max_depth': 14, 'min_samples_split': 2, 'min_samples_leaf': 17}. Best is trial 0 with value: 0.6842323079045386.
[I 2025-08-27 20:25:05,108] Trial 2 finished with value: 0.624775831149124 and parameters: {'n_estimators': 59, 'max_depth': 6, 'min_samples_split': 9, 'min_samples_leaf': 8}. Best is trial 0 with value: 0.6842323079045386.
[I 2025-08-27 20:25:14,474] Trial 3 finished with value: 0.6865774589598566 and parameters: {'n_estimators': 79, 'max_depth': 16, 'min_samples_split': 20, 'min_samples_leaf': 1}. Best is trial 3 with value: 0

🏃 View run RandomForest_SMOTE_TFIDF_Trigrams_Corrected at: https://dagshub.com/satyajeetrai007/Youtube-Comment-Sentiment-Analysis.mlflow/#/experiments/6/runs/c507e5d5955440c48e9cacba043218c3
🧪 View experiment at: https://dagshub.com/satyajeetrai007/Youtube-Comment-Sentiment-Analysis.mlflow/#/experiments/6
